In [1]:
"""
Copyright (C) 2022 Dominik Lentrodt

This file is part of pyrot.

pyrot is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

pyrot is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with pyrot.  If not, see <http://www.gnu.org/licenses/>.
"""

%matplotlib inline

### Import packages

In [2]:
# for calculations
import numpy as np
import scipy as sp
from scipy.signal import argrelextrema

# for plotting
import matplotlib
import matplotlib.pylab as plt

In [3]:
### for development ###
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pyrot

ImportError: attempted relative import with no known parent package

In [ ]:
# ### if using installed distribution ###
# import pyrot

### Example system calculation

Define system

In [ ]:
tThin = 0.01 #thin layer
N = [1.0,   10.0, 1.0,   10.0, 1.0] # Refractive index of each layer
T = [ -1, tThin, 1.0, tThin,  -1] # Thicknesses of each layer

k = np.linspace(0.001, 15., 200000) # incident frequency / wavenumber

In [ ]:
### atom parameters ###
atom_om  = 3.623
atom_om2 = 6.536
atom_om3 = 9.564
atoms_params = [[0.61, 0.01, atom_om,  0.0],
                #[0.61, 0.01, atom_om2, 0.0],
                [0.61, 0.01, atom_om3, 0.0],
               ]

In [ ]:
cavity = pyrot.Cavity1d(N, T)
atoms  = pyrot.CavityAtoms1d(N, T, atoms_params)

Calculate **empty cavity** scattering

In [ ]:
%%time
R_empty = cavity.reflection_intensity(k)
T_empty = cavity.transmission_intensity(k)

Calculate reflection minimum

In [ ]:
max_inds = argrelextrema(R_empty, np.less)
ref_min  = k[max_inds[0][1]]+0.000001 # small number to avoid divide by zeroes
ref_min2 = k[max_inds[0][2]]+0.000001 # small number to avoid divide by zeroes
ref_min3 = k[max_inds[0][3]]+0.000001 # small number to avoid divide by zeroess
print(ref_min)
print(ref_min2)
print(ref_min3)

Calculate cavity scattering **with atom**

In [ ]:
%%time
R_atoms = atoms.linear_reflection_intensity(k)
print("###")
T_atoms = atoms.linear_transmission_intensity(k)

Print linear refractive index with atom delta approximated as thin layer

In [ ]:
n_int, t_int = atoms.linear_layer_system_with_atom(k)
print(n_int)
print(t_int)

### Plot

In [ ]:
t_tot = np.sum(T[1:-1])
outside_buffer = 0.2
depth = np.linspace(-outside_buffer, t_tot+outside_buffer, 10000)

#cavity.draw_cav(depth)
atoms.draw_cav(depth)

In [ ]:
### plot meta params ###
resonant_layer_idx = 3

zoom_dif  = 0.05
zoom_dif2 = 0.05

zoom_dif_ref  = 0.002
zoom_dif2_ref = 0.002
ylim_ref = [-0.5e5, 0.5e5]

In [ ]:
### reflection ###
plt.figure(figsize=(6*3,4))

plt.subplot(131, xlabel='Incident frequency [arb. units]', ylabel='Reflectance',
            title='Reflectance of cavity')
plt.plot(k, R_atoms, label='with atom')
plt.plot(k, R_empty, '--', label='empty cavity')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')
plt.axvline(atom_om3, color='gray', dashes=[3,3], lw=1.0, label='atom transition frequency 2')
plt.legend(loc=1, fontsize=8)

plt.subplot(132, xlabel='Incident frequency [arb. units]', ylabel='Reflectance',
            title='Reflectance of cavity (zoom around atom frequency)')
plt.plot(k, R_atoms, label='with atom')
plt.plot(k, R_empty, '--', label='empty cavity')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')

plt.xlim([atom_om-zoom_dif, atom_om+zoom_dif])
plt.legend(loc=1, fontsize=8)

plt.subplot(133, xlabel='Incident frequency [arb. units]', ylabel='Reflectance',
            title='Reflectance of cavity (zoom around atom frequency 2)')
plt.plot(k, R_atoms, label='with atom')
plt.plot(k, R_empty, '--', label='empty cavity')
plt.axvline(atom_om3, color='gray', dashes=[3,3], lw=1.0, label='atom transition frequency 2')

plt.xlim([atom_om3-zoom_dif2, atom_om3+zoom_dif2])
plt.legend(loc=1, fontsize=8)

plt.show()

### transmission ###
plt.figure(figsize=(6*3,4))

plt.subplot(131, xlabel='Incident frequency [arb. units]', ylabel='Transmittance',
            title='Transmittance of cavity')
plt.plot(k, T_atoms, label='with atom')
plt.plot(k, T_empty, '--', label='empty cavity')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')
plt.axvline(atom_om3, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency 2')
#plt.legend(loc=4, fontsize=8)

plt.subplot(132, xlabel='Incident frequency [arb. units]', ylabel='Transmittance',
            title='Transmittance of cavity (zoom around atom frequency)')
plt.plot(k, T_atoms, label='with atom')
plt.plot(k, T_empty, '--', label='empty cavity')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')

plt.xlim([atom_om-zoom_dif, atom_om+zoom_dif])
#plt.legend(loc=4, fontsize=8)

plt.subplot(133, xlabel='Incident frequency [arb. units]', ylabel='Transmittance',
            title='Transmittance of cavity (zoom around atom frequency 2)')
plt.plot(k, T_atoms, label='with atom')
plt.plot(k, T_empty, '--', label='empty cavity')
plt.axvline(atom_om3, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency 2')

plt.xlim([atom_om3-zoom_dif2, atom_om3+zoom_dif2])
#plt.legend(loc=4, fontsize=8)

plt.show()

In [ ]:
### linear refractive index of atom layer
### Note that without an intrinsic gamma, this curve does not have a Lorentzian width!
plt.figure(figsize=(6*3,4))

plt.subplot(131, xlabel='Incident frequency [arb. units]', ylabel='n',
            title='Refractive index of atomic layer')
plt.plot(k, np.real(n_int[resonant_layer_idx]), label='Re[n]')
plt.plot(k, np.imag(n_int[resonant_layer_idx]), label='Im[n]')
plt.plot(k, np.abs(n_int[resonant_layer_idx]), label='Abs[n]')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')
plt.axvline(atom_om3, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency 2')
plt.ylim(ylim_ref)

#plt.legend(loc=4, fontsize=8)

plt.subplot(132, xlabel='Incident frequency [arb. units]', ylabel='n',
            title='Refractive index of atomic layer (zoom 1)')
plt.plot(k, np.real(n_int[resonant_layer_idx]), label='Re[n]')
plt.plot(k, np.imag(n_int[resonant_layer_idx]), label='Im[n]')
plt.plot(k, np.abs(n_int[resonant_layer_idx]), label='Abs[n]')
plt.axvline(atom_om, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency')
plt.ylim(ylim_ref)

plt.xlim([atom_om-zoom_dif_ref, atom_om+zoom_dif_ref])
#plt.legend(loc=4, fontsize=8)

plt.subplot(133, xlabel='Incident frequency [arb. units]', ylabel='n',
            title='Refractive index of atomic layer (zoom 2)')
plt.plot(k, np.real(n_int[resonant_layer_idx]), label='Re[n]')
plt.plot(k, np.imag(n_int[resonant_layer_idx]), label='Im[n]')
plt.plot(k, np.abs(n_int[resonant_layer_idx]), label='Abs[n]')
plt.axvline(atom_om3, color='k', dashes=[3,3], lw=1.0, label='atom transition frequency 2')
plt.ylim(ylim_ref)

plt.xlim([atom_om3-zoom_dif2_ref, atom_om3+zoom_dif2_ref])
#plt.legend(loc=4, fontsize=8)

plt.show()